In [ ]:
import sys
sys.path.append('./src')
import pandas as pd
from app.llm import async_embed_text, async_response_openai, GenText
from app.ranker import retrieve_top_k
from app.prompts import Prompts
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## Load texts

In [ ]:
import os

root_dir = "book"   # top-level book directory

book_data = []
for chapter, subdirs, files in os.walk(root_dir):
    if chapter == root_dir:
        continue
    # print(f"\n📂 Chapter: {chapter} ")
    chapter_number = int(chapter.split('\\')[-1].split('__')[0])
    chapter_title = chapter.split('\\')[-1].split('__')[-1]

    for file in files:
        # print(f"$ {file}")
        subchapter_number = file.split('__')[0]
        subchapter_title = file.split('__')[1]
        subchapter_page = int(file.split('__')[-1].split('.')[0])
        
        if file.endswith(".txt"):
            filepath = os.path.join(chapter, file)
            with open(filepath, "r", encoding="utf-8") as f:
                # print(f"\n--- {file} ---")
                subchapter_text = f.read()
            book_data.append(
                {
                    "chapter_number": chapter_number,
                    "chapter_title": chapter_title,
                    "subchapter_number": subchapter_number,
                    "subchapter_title": subchapter_title,
                    "subchapter_page": subchapter_page,
                    "subchapter_text": subchapter_text
                }
            )


In [ ]:
# Export segmented book to parquet
df = pd.DataFrame(book_data)
df.to_parquet("book.parquet", index=False)

## Embeddings

In [ ]:
df_partition = df[df['chapter_number'] == 1]

In [ ]:
text_list = df_partition.subchapter_text.to_list()

In [ ]:
import asyncio

tasks = [
    async_embed_text(text=text)
    for text in text_list
    ]

embeddings = await asyncio.gather(*tasks, return_exceptions=True)

In [ ]:
df_partition_embeddings = df_partition.assign(embedding=embeddings)
df_partition_embeddings.to_parquet('book_partition_1.parquet', index=False)

## Retrieval

In [ ]:
df_rag = pd.read_parquet('book_partition_1.parquet')

In [ ]:
question = """
Cutting a midsagittal section through the body separates which 
parts of the body?
 a. anterior and posterior portions
 b. superior and inferior portions
 c. proximal and distal portions
 d. right and left halves
"""

student_answer = "c. proximal and distal portions"

retrieval_top_k = 1

In [ ]:
question_text_embedding = await async_embed_text(text=question)

In [ ]:
df_rag_ranked = retrieve_top_k(
    df_rag=df_rag, 
    query_embedding=question_text_embedding,
    top_k=retrieval_top_k)

In [ ]:
df_rag_ranked
retrieved_pages = df_rag_ranked['subchapter_page'].values
retrieved_text = " \n".join(df_rag_ranked['subchapter_text'].values)
retrieved_text


## LLM feedback

In [ ]:


system_prompt, user_prompt = await Prompts.feedback(
    question=question,
    student_answer=student_answer,
    retrieved_text=retrieved_text
    )

In [ ]:
response = await async_response_openai(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    model = 'gpt-4o-mini',
    response_model=GenText,
    temperature=0.0001
)

In [ ]:
response.text